In [145]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from random import randint
# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
from geopy.geocoders import Nominatim

from time import sleep
import requests
import time,datetime
import os
%matplotlib inline

In [101]:
findzip = ZipcodeSearchEngine()

In [219]:
historical_data = pd.read_csv('clean_train_data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [220]:
historical_data.head()

,Unnamed: 0,day_interval,distance,dropoff_date,dropoff_datetime,dropoff_hour,dropoff_latitude,dropoff_longitude,dropoff_zipcode,fare_amount,...,surcharge,tip_amount,tolls_amount,total_amount,trip_distance,trip_time_in_secs,vendor_id,waiting_penalty,waiting_time,weather
0,186987.0,2,10.418191,2013-01-13,2013-01-13 20:58:00,20,40.793388,-73.972809,10025,27.0,...,0.5,5.5,4.8,38.3,9.06,1200,VTS,0.000000,0.000000,Rain
1,136205.0,2,0.376856,2013-01-13,2013-01-13 21:10:00,21,40.782097,-73.978867,10024,3.5,...,0.5,0.0,0.0,4.5,0.44,120,VTS,0.029670,21.362104,Rain
2,114412.0,2,0.996570,2013-01-13,2013-01-13 21:35:00,21,40.751007,-73.978104,10168,4.5,...,0.5,0.0,0.0,5.5,0.65,240,VTS,0.691434,497.832494,Rain
3,148407.0,2,4.608048,2013-01-13,2013-01-13 21:46:00,21,40.717236,-74.003563,10278,8.5,...,0.5,1.0,0.0,10.5,2.28,420,VTS,2.600077,1872.055354,Rain
4,114616.0,2,1.231572,2013-01-13,2013-01-13 22:15:00,22,40.739117,-73.987694,10010,4.5,...,0.5,1.0,0.0,6.5,0.91,180,VTS,4.437344,3194.887668,Rain


In [221]:
historical_data.keys()

Index(['Unnamed: 0', 'day_interval', 'distance', 'dropoff_date',
       'dropoff_datetime', 'dropoff_hour', 'dropoff_latitude',
       'dropoff_longitude', 'dropoff_zipcode', 'fare_amount', 'hack_license',
       'medallion', 'mta_tax', 'passenger_count', 'payment_type',
       'pickup_date', 'pickup_datetime', 'pickup_day', 'pickup_hour',
       'pickup_latitude', 'pickup_longitude', 'pickup_minute', 'pickup_time',
       'pickup_weekday', 'pickup_zipcode', 'rate_code', 'speed',
       'store_and_fwd_flag', 'surcharge', 'tip_amount', 'tolls_amount',
       'total_amount', 'trip_distance', 'trip_time_in_secs', 'vendor_id',
       'waiting_penalty', 'waiting_time', 'weather'],
      dtype='object')

## Best Revenue from Historical Data

In [222]:
best_revenue = historical_data.groupby(['hack_license','pickup_date'])['total_amount'].sum()

In [223]:
np.max(best_revenue.reset_index()['total_amount'])

1959.2000000000003

## Preprocess

In [224]:
historical_data["dropoff_minute"] = pd.to_datetime(historical_data["dropoff_datetime"]).apply(lambda x: x.minute)
historical_data["dropoff_time"] =historical_data["dropoff_hour"] + (historical_data["dropoff_minute"] / 60)

## Reinforcement Model

In [225]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        if row['waiting_time']/3600<=2:
            return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
        else:
            return row['total_amount']
        
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            

In [226]:
env = TaxiWorld()

In [227]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0, Q_table_file_name='saved_Q.json'):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
        self.dropPickMap= dict()
        self.Q_table_file_name = Q_table_file_name
        self.load_Q()
        
    def load_Q(self):
        if os.path.isfile(self.Q_table_file_name):
            with open(self.Q_table_file_name, 'r') as fp:
                self.Q = json.load(fp)
    
    def store_Q(self):
        with open(self.Q_table_file_name, 'w') as fp:
            json.dump(self.Q, fp)
        
    def generate_key(self, row):
        return str(row['pickup_zipcode']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval']) + '_' + str(row['weather'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_zipcode': tuples[0],
                   'pickup_weekday': tuples[1],
                   'day_interval': tuples[2],
                   'weather': tuples[3]}
    
    def get_meantriptime(self, data):
        map=historical_data.groupby(['pickup_zipcode', 'dropoff_zipcode'])['trip_time_in_secs'].mean()
        return map.reset_index()
    
    def isValidPickup(self, taxidata, dropPickMap):
        key=self.generate_key(taxidata)
        if key in dropPickMap:
            return True
        else:
            return False
        
    def generate_dropPickMap(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            if key not in self.dropPickMap:
                self.dropPickMap[key] = []
            if str(row['dropoff_zipcode']) not in self.dropPickMap[key]:
                self.dropPickMap[key].append(str(row['dropoff_zipcode']))
        return self.dropPickMap
    
    def getDropoff(self, key, dropPickMap):
        posDropoff= dropPickMap[key]
        print("dropoffs",posDropoff )
        rand=randint(0, len(posDropoff)-1)
        return posDropoff[rand]
         
    def getDayInterval(self,hour):
        if hour>=6 and hour<12:
            return 0
        elif hour>=12 and hour<18:
            return 1
        elif hour>=18 and hour<24:
            return 2
        else:
            return 3
        
    def getAction(self, taxidata, _Q,dropPickMap, const):
        best_reward=0
        #best_key = 0
        s = self.generate_key(taxidata)
        best_key=s
        #print(taxidata)
        if s in _Q:
            best_reward=_Q[s][0]*_Q[s][1]/const
        
        zipcode=findzip.by_zipcode(taxidata['dropoff_zipcode'])
        temp=taxidata
        possible_pickup=[]
        returns=100
        while not possible_pickup:
            pickup_res = findzip.by_coordinate(zipcode['Latitude'], zipcode['Longitude'], radius=10, returns=returns)
            for i in range(len(pickup_res)):
                temp['pickup_zipcode']=pickup_res[i]['Zipcode']
                #print(temp['pickup_zipcode'])
                if self.isValidPickup(temp, dropPickMap):
                    possible_pickup.append(pickup_res[i]['Zipcode'])

            print("valid pickup", possible_pickup)
            returns+=10
        
        for key in _Q:
            tuples=self.get_tuples(key)
               
            
            if tuples['pickup_zipcode'] in possible_pickup and tuples['weather']==str(taxidata['weather']) and tuples['pickup_weekday']==str(taxidata['pickup_weekday']) and tuples['day_interval']==str(taxidata['day_interval']):
            
                tempReward=_Q[key][0]*_Q[key][1]/const
                
                if tempReward>best_reward:
                    #print("in temp reward: ",tempReward)
                    #print("in best reward: ",best_reward)
                    best_key=key
                    best_reward= tempReward
                    #print(self.get_tuples(best_key))
                    
        return best_key
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare, Q_table_file_name =  'saved_Q.json'):
        self.Q_table_file_name = Q_table_file_name
        self.build_Q_table(trip_and_fare)
        self.store_Q()
        return self.Q
        
    def distance(self, lat1, lon1, lat2, lon2):
        """calculates the Manhattan distance between 2 points
            using their coordinates

        Parameters
        ----------
        lat1: float
            latitude of first point

        lon1: float
            longitude of first point

        lat2: float
            latitude of second point

        lon2: float
            longitude of second point

        Returns
        -------
        d: float
            The Manhattan distance between the two points in kilometers

        """

        d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
        return d
    
    def find_distance(self, zip1, zip2):
        zipcode1=findzip.by_zipcode(zip1)
        zipcode2=findzip.by_zipcode(zip2)
        dis=self.distance(zipcode1['Latitude'], zipcode1['Longitude'],zipcode2['Latitude'], zipcode2['Longitude'])
        return dis
    
    
    def test(self, taxidata, _Q, dropPickMap,meanTrip, maxstep=1000):
        # TODO: Add your test routine that exmines learned policy
        #      Using the learned policy, generate the trajectory of 
        curr_dropoff_hour=taxidata['dropoff_time']
        #curr_pickup=taxidata['dropoff_time']
        end_time=curr_dropoff_hour+ 12
        
        trace = []
        rewards = []
        
        while curr_dropoff_hour <= end_time:
            
            best_key=self.getAction( taxidata, _Q,dropPickMap, 1000)
            pickup_tuple= self.get_tuples(best_key)
            
            if int(pickup_tuple['pickup_zipcode'])!=taxidata['dropoff_zipcode']:
                dis=self.find_distance(taxidata['dropoff_zipcode'], int(pickup_tuple['pickup_zipcode']))
                curr_dropoff_hour+= dis/taxidata['speed']
                print("roaming time", dis/taxidata['speed'] )
                
            dropoff=self.getDropoff(best_key, dropPickMap)
                
            print("going from ",pickup_tuple['pickup_zipcode'], " to ",  dropoff, " at ", curr_dropoff_hour)
            #curr_dropoff=curr_dropoff+meanTrip[meanTrip['pickup_zipcode']]
            #print(pickup_tuple['pickup_zipcode'])
            temptrip = meanTrip[(meanTrip['pickup_zipcode']==int(pickup_tuple['pickup_zipcode'])) & (meanTrip['dropoff_zipcode']==int(dropoff))]
            meanTripTime= temptrip.iloc[0]['trip_time_in_secs']
            #print(meanTripTime)
            
            curr_dropoff_hour=curr_dropoff_hour+meanTripTime/3600
            taxidata['dropoff_zipcode']=int(dropoff)
            taxidata['day_interval']=self.getDayInterval(curr_dropoff_hour)

## Train the Model

In [228]:
taxiRevenue = TaxiRevenue(env, 5.0, 'saved_Q.json')

In [229]:
_Q = taxiRevenue.train(historical_data)

In [230]:
len(_Q)

3155

## Generate Test Data

In [231]:
dropPickMap=taxiRevenue.generate_dropPickMap(historical_data)
meanTrip= taxiRevenue.get_meantriptime(historical_data)

## Test the Model

In [232]:
taxiRevenue.test(historical_data.iloc[14000], _Q, dropPickMap, meanTrip)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


valid pickup ['10036', '10018', '10001', '10020', '10019', '10199', '10119', '10112', '10111', '10173', '10165', '10069', '10168', '10167', '10170', '10154', '10016', '10174', '10022', '10011', '10023', '10017', '10010', '10065', '10014', '10003', '10021', '10044', '10024', '10075', '11109', '10009', '10028', '10012', '10128', '10013', '11101', '10025', '11222', '10278', '10282', '10002', '10007', '10029', '11106', '10103', '11102', '10026', '10006', '10038', '10280', '10115', '11104', '11211', '10027', '10004', '10035', '10005', '11201', '11103', '11206', '11105', '10037', '10030', '11377', '11205', '11378', '10031', '11370', '10451', '11237', '11217']
roaming time 0.09111998980611906


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


dropoffs ['10167', '10022', '10173', '10010', '10023', '10112', '10028', '10020', '10016', '10001', '10111', '10165', '10119', '10021', '10018', '10065', '10154', '10012', '10019', '10282', '10004', '10014', '10003', '10009', '10036', '10011', '10006', '10278', '10168', '10013', '11364', '11371', '10174', '10115', '10025', '10170', '10075', '10017', '10007', '10128', '10199', '10029', '11369', '10024', '11222', '10032', '10039', '11430', '10280', '10002', '11215', '11201', '11368', '10035', '10463', '10038', '10069', '10030', '11225', '11378', '11377', '11366', '7114', '10453', '10031', '11101', '10005', '10027', '11104', '11239', '11234', '11209', '11373', '11102', '11375', '11233', '10452', '10044', '11237', '11221', '10026', '10033', '10040', '11211', '11224', '11217', '11213', '11219', '10457', '11216', '10037', '11206', '11231', '10461', '11205', '11106', '10462', '10454']
going from  10119  to  10022  at  7.391119989806119
valid pickup ['10022', '10154', '10111', '10167', '10112'

roaming time 0.17378334758452288
dropoffs ['10167', '10022', '10173', '10010', '10023', '10112', '10028', '10020', '10016', '10001', '10111', '10165', '10119', '10021', '10018', '10065', '10154', '10012', '10019', '10282', '10004', '10014', '10003', '10009', '10036', '10011', '10006', '10278', '10168', '10013', '11364', '11371', '10174', '10115', '10025', '10170', '10075', '10017', '10007', '10128', '10199', '10029', '11369', '10024', '11222', '10032', '10039', '11430', '10280', '10002', '11215', '11201', '11368', '10035', '10463', '10038', '10069', '10030', '11225', '11378', '11377', '11366', '7114', '10453', '10031', '11101', '10005', '10027', '11104', '11239', '11234', '11209', '11373', '11102', '11375', '11233', '10452', '10044', '11237', '11221', '10026', '10033', '10040', '11211', '11224', '11217', '11213', '11219', '10457', '11216', '10037', '11206', '11231', '10461', '11205', '11106', '10462', '10454']
going from  10119  to  10278  at  12.028177370849889
valid pickup ['10278', 

roaming time 0.23997689494651808
dropoffs ['10018', '10017', '10036', '10119', '10016', '10001', '10021', '10006', '10022', '10010', '10014', '10020', '10111', '10024', '10012', '10009', '10013', '10019', '10128', '10173', '10003', '10112', '10065', '11369', '10031', '11371', '10038', '10011', '10278', '10168', '11211', '10005', '10007', '10167', '10028', '10023', '10174', '10004', '10282', '10165', '10025', '10069', '10154', '11101', '11218', '10002', '11238', '10199', '7114', '10472', '11430', '10075', '11201', '10029', '10032', '10033', '10115', '11359', '10456', '10044', '11215', '11102', '11231', '11109', '11203', '11220', '10170', '10467', '10026', '11217', '10280', '10030', '11212', '10035', '11235', '11237', '11373', '11222', '11435', '11228', '11372', '11206', '11233', '11210', '11205', '11412', '11106', '10037', '11411', '10103', '11230', '11221', '10453', '11213', '11216', '11436', '7302', '11225', '11375', '11234']
going from  10119  to  10017  at  17.728331671592372
valid 

## Merge Train Data

In [206]:
train_data1 = pd.read_csv('clean_data.csv')
train_data2 = pd.read_csv('clean_data_2.csv')
train_data3 = pd.read_csv('clean_data_3.csv')
frames = [train_data1, train_data2, train_data3]
result = pd.concat(frames)
result.to_csv('clean_train_data.csv', index=False)

In [218]:
result.shape

(966757, 38)